<a href="https://colab.research.google.com/github/ferragina/MyInformationRetrieval/blob/main/6_LLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **LLM in practice**

In [ ]:
!pip install transformers>=4.41.2 accelerate>=0.31.0

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# import the required classes
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [ ]:
# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("../models/microsoft/Phi-3-mini-4k-instruct")

model = AutoModelForCausalLM.from_pretrained(
    "../models/microsoft/Phi-3-mini-4k-instruct",
    device_map="cpu",
    torch_dtype="auto",
    trust_remote_code=True,
)

# **Pipeline**

Now you can wrap the model and the tokenizer in a pipeline object that has "text-generation" as task.

In [ ]:
# Create a pipeline
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False, # False means to not include the prompt text in the returned text
    max_new_tokens=50,
    do_sample=False, # no randomness in the generated text
)

# **Generating a Text Response to a Prompt**

You'll now use the pipeline object (labeled as generator) to generate a response consisting of 50 tokens to the given prompt.

In [ ]:
prompt = "Write an email apologizing to Sarah for the tragic gardening mishap. Explain how it happened. "

output = generator(prompt)

print(output[0]['generated_text'])

# **Exploring the Model's Architecture**

In [ ]:
model

In [ ]:
model.model.embed_tokens

In [ ]:
model.model

In [ ]:
model.model.layers[0]

Generating a Single Token to a Prompt
You earlier used the Pipeline object to generate a text response to a prompt. The pipeline provides an abstraction to the underlying process of text generation. Each token in the text is actually generated one by one.

Let's now give the model a prompt and check the first token it will generate.

prompt = "The capital of France is"
You'll need first to tokenize the prompt and get the ids of the tokens.

# Tokenize the input prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
input_ids
Let's now pass the token ids to the transformer block (before the LM head).

# Get the output of the model before the lm_head
model_output = model.model(input_ids)
The transformer block outputs for each token a vector of size 3072 (embedding size). Let's check the shape of this output.

# Get the shape the output the model before the lm_head
model_output[0].shape
The first number represents the batch size, which is 1 in this case since we have one prompt. The second number 5 represents the number of tokens. And finally 3072 represents the embedding size (the size of the vector that corresponds to each token).

Let's now get the output of the LM head.

# Get the output of the lm_head
lm_head_output = model.lm_head(model_output[0])
lm_head_output.shape
The LM head outputs for each token in the input prompt, a vector of size 32064 (vocabulary size). So there are 5 vectors, each of size 32064. Each vector can be mapped to a probability distribution, that shows the probability for each token in the vocabulary to come after the given token in the input prompt.

Since we're interested in generating the output token that comes after the last token in the input prompt ("is"), we'll focus on the last vector. So in the next cell, lm_head_output[0,-1] is a vector of size 32064 from which you can generate the token that comes after ("is"). You can do that by finding the id of the token that corresponds to the highest value in the vector lm_head_output[0,-1] (using argmax(-1), -1 means across the last axis here).

token_id = lm_head_output[0,-1].argmax(-1)
token_id
Finally, let's decode the returned token id.

tokenizer.decode(token_id)
⬇   Download Notebooks: If you'd like to donwload the notebook: 1) click on the "File" option on the top menu of the notebook and then 2) click on "Download as" and select "Notebook (.ipynb)". For more help, please see the "Appendix – Tips, Help, and Download" Lesson.